In [1]:
import sympy
import sympy.physics.mechanics as mech
sympy.init_printing(use_latex='mathjax')
from IPython.display import display

Define BKE Function

In [2]:
def bke(vector, frame_i, frame_b, t):
    return (vector.diff(t, frame_b) + frame_b.ang_vel_in(frame_i).cross(vector))

Define Symbolic Variables

In [3]:
T, r, m_w, m_p, l, F, x, g, alpha, theta, t, R_x, R_z, N, J_p, J_w, v_x, omega, k_emf, b_damp, V, J_motor, a = \
    sympy.symbols('T r m_w m_p l F x g alpha theta t R_x R_z N J_p J_w v_x omega k_emf b_damp V J_motor a')

# Dynamics

Define Reference Frames

In [4]:
frame_i = mech.ReferenceFrame('i') #inertial frame
frame_b = frame_i.orientnew('b', 'Axis', [theta(t), frame_i.y]) #fixed in pendulum
frame_w = frame_b.orientnew('w', 'Axis', [-alpha(t), frame_i.y]) #fixed in wheel

Define Points of Interest

In [5]:
point_o = mech.Point('o') 
point_o.set_vel(frame_i, 0) #point o is inertially fixed

point_W = point_o.locatenew('W', frame_i.x*x(t))  #wheel c.m.
point_W.set_vel(frame_b, 0) #point W is fixed in pendulum frame, too
point_W.set_vel(frame_i, point_W.pos_from(point_o).diff(t, frame_i))

point_P = point_W.locatenew('P', frame_b.z*(-l)) #pendulum c.m.
point_P.set_vel(frame_b, 0)
point_P.v2pt_theory(point_W, frame_i, frame_b);

Define Bodies

In [6]:
# Wheel Creation
J_wheel = mech.inertia(frame_w, 0, J_w, 0)
wheel = mech.RigidBody('wheel', point_W, frame_w, m_w, (J_wheel, point_W))

# Pendulum Creation
J_pend = mech.inertia(frame_b, 0, J_p, 0)
pend = mech.RigidBody('pend', point_P, frame_b, m_p, (J_pend, point_P)) #change inertia point to point_p

Lagranges Method

In [ ]:
noslip = [(r*(alpha(t) - theta(t)) - x(t))] # constraint equation, same as before?
flist = [(point_P, m_p*g*frame_i.z), (frame_b, T(t)*frame_i.y), (frame_b, b_damp*alpha(t).diff(t)*frame_i.y)] #force list, external torque T, damping b in y direction 
L = mech.Lagrangian(frame_i, wheel, pend)
#l = mech.LagrangesMethod(L, [x(t), theta(t)], forcelist = flist, hol_coneqs=noslip, frame = frame_i)
l = mech.LagrangesMethod(L, [x(t), theta(t), alpha(t)], forcelist = flist, frame = frame_i)
eoms = l.form_lagranges_equations()
eoms

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢                    ⎛    2           2      ⎞         2               2      
⎢                    ⎜   d           d       ⎟        d               d       
⎢                J_w⋅⎜- ───(α(t)) + ───(θ(t))⎟   J_w⋅───(α(t))   J_w⋅───(θ(t))
⎢      2             ⎜    2           2      ⎟      

In [ ]:
eom_sol = sympy.solve(eoms, [theta(t).diff(t,2), alpha(t).diff(t,2), x(t).diff(t,2), \
        alpha(t).diff(t)], simplify=False)
eom_sol

In [44]:
theta_ddot = eom_sol[theta(t).diff(t,2)].expand().ratsimp().collect([theta(t), x(t), theta(t).diff(t), x(t).diff(t)], sympy.factor)
theta_ddot

                                                                              
            ⎛       d                                  ⎞             ⎛        
(m_p + m_w)⋅⎜b_damp⋅──(α(t)) + g⋅l⋅m_p⋅sin(θ(t)) + T(t)⎟⋅cos(θ(t)) + ⎝J_p⋅m_p⋅
            ⎝       dt                                 ⎠                      
──────────────────────────────────────────────────────────────────────────────
                                                                              
                                                                              

                                                                              
                                                                              
sin(θ(t)) - J_p⋅m_p⋅cos(θ(t))⋅tan(θ(t)) + J_p⋅m_w⋅sin(θ(t)) - J_p⋅m_w⋅cos(θ(t)
                                                                              
──────────────────────────────────────────────────────────────────────────────
                                  ⎛                

In [35]:
#l.forcing
l.mass_matrix

⎡   m_p + m_w       -l⋅m_p⋅cos(θ(t))    0  ⎤
⎢                                          ⎥
⎢                               2          ⎥
⎢-l⋅m_p⋅cos(θ(t))  J_p + J_w + l ⋅m_p  -J_w⎥
⎢                                          ⎥
⎣       0                 -J_w         J_w ⎦